In [ ]:
import numpy as np
import skimage.feature as skif
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from PIL import Image

In [ ]:

# Load the ResNet-50 model (pretrained on ImageNet)
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

# Function to extract HOG features from an image
def extract_hog_features(image):
    resized_image = resize(image, (224, 224))  # Resize the image to match ResNet-50 input shape
    preprocessed_image = preprocess_input(resized_image)  # Preprocess the image for ResNet-50
    features = resnet_model.predict(np.expand_dims(preprocessed_image, axis=0))  # Get ResNet-50 features
    
    # Convert features to HOG representation
    hog_features = skif.hog(features.flatten(), orientations=8, pixels_per_cell=(16, 16),
                            cells_per_block=(1, 1))
    
    return hog_features

In [ ]:

# Set the path to the dataset directory
dataset_dir = "path/to/dataset"

# Set the output directory for HOG features
output_dir = "path/to/output"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over images in the dataset directory
for filename in os.listdir(dataset_dir):
    image_path = os.path.join(dataset_dir, filename)
    if os.path.isfile(image_path):
        # Load and process the image
        image = np.array(Image.open(image_path).convert("RGB"))
        
        # Extract HOG features
        hog_features = extract_hog_features(image)
        
        # Save the HOG features as a numpy array
        output_path = os.path.join(output_dir, f"{filename}.npy")
        np.save(output_path, hog_features)
        print(f"Saved HOG features for {filename}")
